In [1]:
import os

In [2]:
%pwd

'c:\\Users\\howar\\Documents\\UIUC\\IE 421\\project\\group_10_project\\mlops\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\howar\\Documents\\UIUC\\IE 421\\project\\group_10_project\\mlops'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    untrained_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_predict_events: int
    params_sequence_length: int
    params_val_interval: int
@dataclass(frozen=True)
class PrepareModelConfig:
    root_dir: Path
    model_path: Path
    params_input_size: int
    params_hidden_size: int
    params_num_layers: int
    params_fc_layers_size: int
    params_learning_rate: float

In [2]:
from src.lstm_price_prediction.constants import *
from src.lstm_price_prediction.utils.common import read_yaml, create_directories
from src.lstm_price_prediction.components.rnn_model import preprocess, training_dataset 
import torch
import pandas as pd

ModuleNotFoundError: No module named 'src.lstm_price_prediction'

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        
    def get_prepare_model_config(self) -> PrepareModelConfig:
        config = self.config.prepare_model
        
        create_directories([config.root_dir])

        prepare_model_config = PrepareModelConfig(
            root_dir=Path(config.root_dir),
            model_path=Path(config.model_path),
            params_input_size = self.params.INPUT_SIZE,
            params_hidden_size = self.params.HIDDEN_SIZE,
            params_num_layers = self.params.NUM_LAYERS,
            params_fc_layers_size = self.params.FC_LAYERS_SIZE,
            params_learning_rate = self.params.LEARNING_RATE
   
        )

        return prepare_model_config
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        params = self.params
        untrained_model = self.config.prepare_model
        training_data_path = "..data/20220801_book_updates.csv"
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            untrained_model_path=Path(untrained_model.model_path),
            trained_model_path=Path(training.trained_model_path),
            training_data=training_data_path,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_sequence_length=params.SEQUENCE_LENGTH,
            params_predict_events=params.PREDICT_EVENTS,
            params_val_interval=params.VAL_INTERVAL
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from src.lstm_price_prediction.components.rnn_model import rnn_models
import torch.nn as nn
from tqdm import tqdm
 

In [9]:
class Training:
    def __init__(self, config: TrainingConfig, model_config: PrepareModelConfig):
        self.config = config
        self.model_config = model_config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        params_filepath = PARAMS_FILE_PATH
        self.params = read_yaml(params_filepath)
        
    
    def get_base_model(self):
        self.model = rnn_models.LSTMModel(self.model_config.params_input_size,self.model_config.params_hidden_size ,self.model_config.params_num_layers, self.model_config.params_fc_layers_size).to(self.device)
        self.model.load_state_dict(torch.load(self.model_config.model_path))
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.model_config.params_learning_rate)
        self.criterion = torch.nn.CrossEntropyLoss()
        
    def train_valid_generator(self):
        
        df = pd.read_csv(self.params.TRAIN_DATA)
        print('Preprocessing data....')
        new_df = preprocess.process_data(df)
        dataset = training_dataset.OrderBookDataset(new_df, self.params.SEQUENCE_LENGTH, self.params.PREDICT_EVENTS)
        print('Data preprocessing complete')
        train_loader, val_loader, test_loader = training_dataset.get_data_loaders(dataset, self.params.TEST_SPLIT, self.params.BATCH_SIZE)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader



    

    
    @staticmethod
    def save_model(path: Path, model: torch.nn.Module):
        torch.save(model.state_dict(), path)



    
    def train(self):
        # Train the model in one epoch
        self.model.train()
        train_loss = 0
        for i, (x, y) in enumerate(tqdm(self.train_loader, desc="Training")):
            x = x.float()  # Convert input data to torch.float32 type
            y = y.long()  # Convert target data to torch.float32 type
            x = x.to(self.device)
            y = y.to(self.device)
            self.optimizer.zero_grad()
            y_hat = self.model(x)
            y_hat = y_hat.to(torch.float32)
            loss = self.criterion(y_hat, y)
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 0.25)
            self.optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(self.train_loader)

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    model_config = config.get_prepare_model_config()
    training = Training(config=training_config, model_config=model_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-05-11 04:40:49,677: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-11 04:40:49,679: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-11 04:40:49,681: INFO: common: created directory at: artifacts]
[2024-05-11 04:40:49,681: INFO: common: created directory at: artifacts\training]
[2024-05-11 04:40:49,682: INFO: common: created directory at: artifacts/prepare_model]
[2024-05-11 04:40:49,684: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-11 04:40:50,254: INFO: file_utils: TensorFlow version 2.12.0 available.]
[2024-05-11 04:40:50,255: INFO: file_utils: PyTorch version 2.1.0 available.]
[2024-05-11 04:40:50,355: INFO: modeling_xlnet: Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .]
Preprocessing data....
Data preprocessing complete


Training: 100%|██████████| 320/320 [00:07<00:00, 43.25it/s]
